In [1]:
import numpy as np
import numba as nb
import pandas as pd
import pyteomics.mgf
import seaborn as sns
from matplotlib import pyplot as plt
import spectrum_utils.spectrum as sus
import sys
import importlib

from tqdm import tqdm

sys.path.append("../../src/")
from ms_similarity_metrics import frequency
from ms_similarity_metrics import similarity_2 as similarity
importlib.reload(frequency)
importlib.reload(similarity)

tqdm.pandas()

## Analysis settings

In [2]:
# Spectra and spectrum pairs to include with the following settings.
charges = 0, 1
min_n_peaks = 6
fragment_mz_tolerance = 0.1
min_mass_diff = 1    # Da
max_mass_diff = 200    # Da

### Load Utils

In [3]:
# Profile spectra contain 0 intensity values.
@nb.njit
def is_centroid(intensity_array):
    return np.all(intensity_array > 0)

# Assumes that the spectra are sorted by ascending precusor m/z.
@nb.njit
def generate_pairs(
    spectrum_indexes, masses, min_mass_diff, max_mass_diff
):
    for i in range(len(spectrum_indexes)):
        j = i + 1
        while (
            j < len(spectrum_indexes) and
            masses[j] - masses[i] < min_mass_diff
        ):
            j += 1
        while (
            j < len(spectrum_indexes) and
            masses[j] - masses[i] < max_mass_diff
        ):
            yield spectrum_indexes[i]
            yield spectrum_indexes[j]
            j += 1

## Data IO

In [4]:
# Read all spectra from the MGF.
# ALL_GNPS_NO_PROPOGATED (retrieved on 2022-05-12) downloaded from
# https://gnps-external.ucsd.edu/gnpslibrary

# Spectrum quality filtering:
#   - Don't include propagated spectra (LIBRARYQUALITY==4).
#   - Don't include multiply charged molecules.
#   - Don't include spectra with invalid precursor m/z (0).
#   - Don't include spectra with too few peaks (minimum 6).
#   - Only include positive ion mode spectra.
#   - Only include spectra with [M+H]+ adducts.
#   - Only include centroid data (does not contain zero intensity values).
#   - Only include spectra with InChI and/or SMILES specified.

spectra = []

# Download from https://zenodo.org/record/6829249/files/ALL_GNPS_NO_PROPOGATED.mgf?download=1
filename = ("../data/ALL_GNPS_NO_PROPOGATED.mgf")

with pyteomics.mgf.MGF(filename) as f_in:
    for spectrum_dict in tqdm(f_in):
        if (
            int(spectrum_dict["params"]["libraryquality"]) <= 3 and
            int(spectrum_dict["params"]["charge"][0]) in charges and
            float(spectrum_dict["params"]["pepmass"][0]) > 0 and
            len(spectrum_dict["m/z array"]) >= min_n_peaks and
            spectrum_dict["params"]["ionmode"] == "Positive" and
            spectrum_dict["params"]["name"].rstrip().endswith(" M+H") and
            is_centroid(spectrum_dict["intensity array"]) and
            (
                spectrum_dict["params"]["inchi"] != "N/A" or
                spectrum_dict["params"]["smiles"] != "N/A"
            )
        ):
            spec = sus.MsmsSpectrum(
                spectrum_dict["params"]["spectrumid"],
                float(spectrum_dict["params"]["pepmass"][0]),
                # Re-assign charge 0 to 1.
                max(int(spectrum_dict["params"]["charge"][0]), 1),
                spectrum_dict["m/z array"],
                spectrum_dict["intensity array"]/max(spectrum_dict["intensity array"]),
            )
            spec.library = spectrum_dict["params"]["organism"]
            spec.inchi = spectrum_dict["params"]["inchi"]
            spec.smiles = spectrum_dict["params"]["smiles"]
            spec.remove_precursor_peak(0.1, "Da")
            spec.filter_intensity(0.01, max_num_peaks=200)
            spectra.append(spec)


0it [00:00, ?it/s]

495600it [01:12, 6824.21it/s] 


In [5]:
# Round spectra m/z to one decimal
for s in spectra:
    s.mz.round(1, out=s.mz)

This takes a bit... (3 min in my laptop)

### Load the metadata

In [6]:
metadata = pd.read_csv(
    'https://zenodo.org/record/6829249/files/gnps_libraries_metadata.csv?download=1'
)

## Get pairs

In [8]:
# Get pair subset that Wout used
# Download file from s3://enveda-data-user/chloe.engler/cosine_similarity/Wout_data/pairs_subset.txt
with open('../../data/pairs_subset.txt', 'r') as f:
    pairs_subset = f.read().splitlines()
    pairs_subset = pairs_subset[1:]
    pairs_subset = [np.array(pair.split(' ')).astype(int) for pair in pairs_subset]
pairs_subset = np.array(pairs_subset)

In [9]:
# Get dataframe containing m/z frequency information
frequency_df = pd.read_csv('s3://enveda-data-user/chloe.engler/cosine_similarity/Wout_data/frequency_df.csv')

# Get number of spectra used to generate the frequency_df
# Get file from s3://enveda-data-user/chloe.engler/cosine_similarity/Wout_data/num_spectra.txt
with open('../../data/num_spectra.txt', 'r') as f:
    num_spectra = int(f.read())

In [10]:
# Get frequency counts
# Get file from s3://enveda-data-user/chloe.engler/cosine_similarity/Wout_data/frequency_count.txt
with open('../../data/frequency_count.txt', 'r') as f:
    frequency_count = f.read().splitlines()
    frequency_count = frequency_count[1:]
    frequency_count = [float(x) for x in frequency_count]

In [11]:
# np.random.seed(1)
#pairs = pairs_subset[np.random.choice(pairs_subset.shape[0], 1_000, replace=False)]
pairs = pairs_subset.copy()

In [12]:
# Save core information about the pairs.
similarities_df = pd.DataFrame(
    {
        "pair1": pairs[:, 0],
        "pair2": pairs[:, 1],
        "id1": metadata.loc[pairs[:, 0], "id"].values,
        "id2": metadata.loc[pairs[:, 1], "id"].values,
    }
)

# Get weights

In [13]:
# Define weight functions
#idf_dict = frequency.idf(frequency_df, num_spectra, frequency_col='frequency')
weight_func = lambda x: x**(1/4)
intensity_weight_func = lambda x: x**(1/4)

In [14]:
# Get m/z frequency weight df
weight_df = frequency.get_weights(frequency_df, weight_func, weight_col='prob')
weight_df = weight_df.set_index('mz', drop=True)
weight_df.head()

,Unnamed: 0,frequency,prob,weight
mz,,,,
4.6,0,1,4.805324e-07,0.026329
9.0,1,1,4.805324e-07,0.026329
10.2,2,1,4.805324e-07,0.026329
14.2,3,1,4.805324e-07,0.026329
15.0,4,6,2.883195e-06,0.041207


# Calculate similarities

In [15]:
# To run this code you need to import the functions from https://github.com/YuanyueLi/SpectralEntropy/tree/master/spectral_entropy (not packaged)
# Simply copy the spectral_entropy.py file in the same folder as this notebook
import spectral_entropy

In [17]:
# Create lists
metrics = []
weighted_metrics = []

for i, j in tqdm(pairs, total=pairs.shape[0]):

    # Add unweighted results
    metrics.append(
        spectral_entropy.all_similarity(
            np.array(list(zip(list(spectra[i].mz), list(spectra[i].intensity))), dtype=np.float32),
            np.array(list(zip(list(spectra[j].mz), list(spectra[j].intensity))), dtype=np.float32),
            ms2_da=0.1,  # Same as Wout's paper
        )
    )
    
    # Add weighted results
    weighted_intensity1 = intensity_weight_func(spectra[i].intensity)*weight_df.loc[spectra[i].mz, 'weight']
    weighted_intensity2 = intensity_weight_func(spectra[j].intensity)*weight_df.loc[spectra[j].mz, 'weight']
    current_dict = spectral_entropy.all_similarity(
            np.array(list(zip(list(spectra[i].mz), 
                              list(weighted_intensity1))), dtype=np.float32),
            np.array(list(zip(list(spectra[j].mz), 
                              list(weighted_intensity2))), dtype=np.float32),
            ms2_da=0.1,  # Same as Wout's paper 
        )
    renamed_dict = {'adjusted_'+key:current_dict[key] for key in current_dict.keys()}
    weighted_metrics.append(
        renamed_dict
    )

 55%|█████▍    | 5482479/10000000 [3:41:20<2:21:02, 533.84it/s]   

: 

: 

In [ ]:
# Concatenate both dataframes.
final_df = pd.concat(
    [
        similarities_df,
        pd.DataFrame(metrics),
    ],
    axis=1,
)

final_weighted_df = pd.concat(
    [
        similarities_df,
        pd.DataFrame(weighted_metrics)
    ],
    axis=1,
)

In [ ]:
# Export dataframe as parquet
final_df.to_parquet("../../data/benchmark_metrics.parquet")
final_weighted_df.to_parquet('../../data/weighted_benchmark_metrics.parquet')